In [33]:
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_chroma import Chroma
from langchain_community.document_loaders import JSONLoader
from IPython.display import display, Markdown

In [22]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.prompts import PromptTemplate

In [20]:
from dotenv import load_dotenv
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.tools.retriever import create_retriever_tool
from langgraph.checkpoint.sqlite import SqliteSaver


In [3]:
_ = load_dotenv()
google_api_key = os.getenv("GOOGLE_API_KEY")
tavily_api_key = os.getenv("TAVILY_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")

In [4]:
model=ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=google_api_key, verbose=True)
google_embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

In [36]:
loader = JSONLoader(file_path="../data/logs.json", jq_schema=".[]", text_content=False)
documents = loader.load()
db = Chroma.from_documents(documents, google_embeddings)

In [37]:
retriever = db.as_retriever()

In [38]:
log_tool = create_retriever_tool(
    retriever,
    "logs_knowledge_base",
    "Search and return information about logs",
)

#### Create the agent

In [23]:
memory = SqliteSaver.from_conn_string(":memory:")

In [17]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [18]:
class Agent:
    def __init__(self, model, tools, checkpointer, system=""):
        self.system = system
        graph=StateGraph(AgentState)
        graph.add_node("llm", self.call_model)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm",
                                    self.exists_action,
                                    {True: "action", False: END})   
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph=graph.compile(checkpointer=checkpointer)
        self.tools = {t.name: t for t in tools}
        self.model= model.bind_tools(tools)

    def call_model(self, state: AgentState):
        messages = state["messages"]
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message=self.model.invoke(messages)
        return {'messages':[message]}
    def exists_action(self, state: AgentState):
        result=state["messages"][-1]
        return len(result.tool_calls) > 0
    
    def take_action(self, state: AgentState):
        tool_calls=state["messages"][-1].tool_calls
        results=[]
        for t in tool_calls:
            print(f"calling tool : {t}")
            result = self.tools[t['name']].invoke(t["args"])
            results.append(ToolMessage(tool_call_id=t['id'],name=t['name'], content=str(result)))
        print("Back to the model")
        return {'messages':results}

##### laod system prompt

In [39]:
with open("../prompts/system_prompt.txt","r") as f:
    system_message = f.read()

tools = [log_tool]
abot = Agent(model, tools, memory, system_message)

In [41]:
user_message= "Describe to me the overall health of the system"
messages = [HumanMessage(content=user_message)]

In [42]:
result= abot.graph.invoke({"messages":messages },
                     config = {"configurable":{"thread_id": "1"}})

calling tool : {'name': 'logs_knowledge_base', 'args': {'query': 'system health'}, 'id': '007c03c1-70e4-4756-af38-5057b5690bd4'}
Back to the model


In [43]:
analysis=result['messages'][-1].content
display(Markdown(analysis))

The system is showing some signs of strain. Recent logs indicate high CPU usage (85%) and memory usage (70%). Additionally, there were warnings logged about CPU temperature exceeding safe limits on web-02 and high memory usage on server web-01. While the system is currently operational, it's recommended to investigate these issues to prevent potential performance degradation or system instability. 


In [44]:
result

{'messages': [HumanMessage(content='Show me all the log entries for user john_doe'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'logs_knowledge_base', 'arguments': '{"query": "log entries for user john_doe"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-de07b3c8-1c4d-4c3d-80f3-cea506bf5018-0', tool_calls=[{'name': 'logs_knowledge_base', 'args': {'query': 'log entries for user john_doe'}, 'id': '850a8f25-2ccd-48a7-a30a-4acb9e46c9c5'}], usage_metadata={'input_tokens': 476, 'output_tokens': 23, 'total_tokens